In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

In [2]:
# Model definition
class CIFAR10_MLP(nn.Module):
    def __init__(self, input_dim=3*32*32, hidden_dims=[2048, 1024, 512, 256], num_classes=10, dropout=0.5):
        super().__init__()
        layers = []
        prev_dim = input_dim
        for hdim in hidden_dims:
            layers.append(nn.Linear(prev_dim, hdim))
            layers.append(nn.BatchNorm1d(hdim))
            layers.append(nn.ReLU(inplace=True))
            layers.append(nn.Dropout(dropout))
            prev_dim = hdim
        layers.append(nn.Linear(prev_dim, num_classes))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # flatten
        return self.net(x)

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data transforms: normalize CIFAR-10 images
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), 
                         (0.247, 0.243, 0.261))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), 
                         (0.247, 0.243, 0.261))
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)


Files already downloaded and verified
Files already downloaded and verified


In [4]:
model = CIFAR10_MLP().to(device)
model = torch.jit.script(model)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [5]:
from utils import train

train_metrics, val_metrics, test_metrics = train(model, train_loader, None, test_loader, 30, optimizer, criterion)


Epoch: 1 Total_Time: 1.8318 Average_Time_per_batch: 0.0047 Train_Accuracy: 0.3389 Train_Loss: 1.8259 
Epoch: 2 Total_Time: 1.4124 Average_Time_per_batch: 0.0036 Train_Accuracy: 0.4258 Train_Loss: 1.6114 
Epoch: 3 Total_Time: 1.4525 Average_Time_per_batch: 0.0037 Train_Accuracy: 0.4552 Train_Loss: 1.5294 
Epoch: 4 Total_Time: 1.5381 Average_Time_per_batch: 0.0039 Train_Accuracy: 0.4751 Train_Loss: 1.4760 
Epoch: 5 Total_Time: 1.4989 Average_Time_per_batch: 0.0038 Train_Accuracy: 0.4925 Train_Loss: 1.4283 
Epoch: 6 Total_Time: 1.5655 Average_Time_per_batch: 0.0040 Train_Accuracy: 0.5060 Train_Loss: 1.3930 
Epoch: 7 Total_Time: 1.4831 Average_Time_per_batch: 0.0038 Train_Accuracy: 0.5181 Train_Loss: 1.3593 
Epoch: 8 Total_Time: 1.2191 Average_Time_per_batch: 0.0031 Train_Accuracy: 0.5284 Train_Loss: 1.3310 
Epoch: 9 Total_Time: 1.4607 Average_Time_per_batch: 0.0037 Train_Accuracy: 0.5406 Train_Loss: 1.3025 
Epoch: 10 Total_Time: 1.4466 Average_Time_per_batch: 0.0037 Train_Accuracy: 0.545

In [6]:
from dpn_3.dpn import DPN as DPN_3
    
model_3 = DPN_3(3*32*32, 100, 10, True).cuda()
#model_3.compile()

In [7]:
model_3 = torch.jit.trace(model_3, torch.randn(128, 3*32*32).cuda())
optimizer = optim.Adam(model_3.parameters())
criterion = nn.CrossEntropyLoss()

In [8]:
train_metrics_3, val_metrics_3, test_metrics_3 = train(model_3, train_loader, None, test_loader, 30, optimizer, criterion)


Epoch: 1 Total_Time: 0.9127 Average_Time_per_batch: 0.0023 Train_Accuracy: 0.3932 Train_Loss: 1.7763 
Epoch: 2 Total_Time: 0.7810 Average_Time_per_batch: 0.0020 Train_Accuracy: 0.4511 Train_Loss: 1.6003 
Epoch: 3 Total_Time: 0.6335 Average_Time_per_batch: 0.0016 Train_Accuracy: 0.4834 Train_Loss: 1.5190 
Epoch: 4 Total_Time: 0.7269 Average_Time_per_batch: 0.0019 Train_Accuracy: 0.5061 Train_Loss: 1.4398 
Epoch: 5 Total_Time: 0.7443 Average_Time_per_batch: 0.0019 Train_Accuracy: 0.5210 Train_Loss: 1.4011 
Epoch: 6 Total_Time: 0.6721 Average_Time_per_batch: 0.0017 Train_Accuracy: 0.5306 Train_Loss: 1.3735 
Epoch: 7 Total_Time: 0.7428 Average_Time_per_batch: 0.0019 Train_Accuracy: 0.5445 Train_Loss: 1.3401 
Epoch: 8 Total_Time: 0.6816 Average_Time_per_batch: 0.0017 Train_Accuracy: 0.5610 Train_Loss: 1.2827 
Epoch: 9 Total_Time: 0.6778 Average_Time_per_batch: 0.0017 Train_Accuracy: 0.5719 Train_Loss: 1.2548 
Epoch: 10 Total_Time: 0.7149 Average_Time_per_batch: 0.0018 Train_Accuracy: 0.582

In [9]:
import torch
from torch import nn

hidden_dims = [2048, 1024, 512, 256]
total = sum(hidden_dims)

blocks = len(hidden_dims)
features = 3 * 32 * 32
neural_blocks = []
for dim in hidden_dims:
    std_dev = torch.sqrt(torch.tensor(1 / features)).to(device)
    neural_blocks.append(torch.randn(dim, features).to(device) * std_dev)
    features += dim

feature_blocks = []
features_start = 0
for i in range(len(neural_blocks)):
    features_end = neural_blocks[i].shape[1]
    block = neural_blocks[i][:, features_start:]
    for j in range(i + 1, len(neural_blocks)):
        block = torch.cat((block, neural_blocks[j][:, features_start:features_end]), dim=0)
    feature_blocks.append(nn.Parameter(block))
    features_start = features_end

biases = biases = nn.Parameter(torch.empty(total).uniform_(0.0, 1.0)).to(device)

In [11]:
from dpn_2.dpn import DPN as DPN_2
    
model_2 = DPN_2(3*32*32, 500, 10, True).cuda()
model_2.weights.extend(feature_blocks)
model_2.biases = biases

In [12]:
#model_3 = torch.jit.trace(model_3, torch.randn(128, 3*32*32).cuda())
optimizer = optim.Adam(model_2.parameters())
criterion = nn.CrossEntropyLoss()

In [13]:
from utils import train
train_metrics_3, val_metrics_3, test_metrics_3 = train(model_2, train_loader, None, test_loader, 50, optimizer, criterion)


Epoch: 1 Total_Time: 1.5261 Average_Time_per_batch: 0.0039 Train_Accuracy: 0.3847 Train_Loss: 2.1045 
Epoch: 2 Total_Time: 1.2521 Average_Time_per_batch: 0.0032 Train_Accuracy: 0.4489 Train_Loss: 1.7124 
Epoch: 3 Total_Time: 1.5861 Average_Time_per_batch: 0.0041 Train_Accuracy: 0.4774 Train_Loss: 1.6420 
Epoch: 4 Total_Time: 1.3246 Average_Time_per_batch: 0.0034 Train_Accuracy: 0.5030 Train_Loss: 1.5923 
Epoch: 5 Total_Time: 1.4549 Average_Time_per_batch: 0.0037 Train_Accuracy: 0.5243 Train_Loss: 1.5507 
Epoch: 6 Total_Time: 1.2767 Average_Time_per_batch: 0.0033 Train_Accuracy: 0.5440 Train_Loss: 1.5052 
Epoch: 7 Total_Time: 1.3041 Average_Time_per_batch: 0.0033 Train_Accuracy: 0.5674 Train_Loss: 1.4457 
Epoch: 8 Total_Time: 1.4317 Average_Time_per_batch: 0.0037 Train_Accuracy: 0.5884 Train_Loss: 1.4063 
Epoch: 9 Total_Time: 1.3680 Average_Time_per_batch: 0.0035 Train_Accuracy: 0.6086 Train_Loss: 1.3488 
Epoch: 10 Total_Time: 1.3129 Average_Time_per_batch: 0.0034 Train_Accuracy: 0.626

In [14]:
class ResNetMLP(nn.Module):
    def __init__(self, num_classes=10, mlp_hidden=256):
        super().__init__()
        # Load ResNet18 (pretrained=False for CIFAR10, since ImageNet weights use 224x224)
        backbone = torchvision.models.resnet18(weights=None)
        # Change input conv layer for CIFAR-10 (3x32x32)
        backbone.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        backbone.maxpool = nn.Identity()  # Remove the first maxpool
        # Extract up to the last layer
        self.backbone = nn.Sequential(*list(backbone.children())[:-1])  # Remove FC
        self.mlp = nn.Sequential(
            nn.Linear(512, mlp_hidden),
            nn.BatchNorm1d(mlp_hidden),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(mlp_hidden, num_classes)
        )

    def forward(self, x):
        x = self.backbone(x)  # [B, 512, 1, 1]
        x = x.view(x.size(0), -1)  # Flatten
        x = self.mlp(x)
        return x

In [15]:
# --- 1. Data Augmentation and Loading ---
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.247, 0.243, 0.261))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.247, 0.243, 0.261))
])


In [16]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)


Files already downloaded and verified
Files already downloaded and verified


In [20]:
model = ResNetMLP(num_classes=10, mlp_hidden=256).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

In [21]:
train_metrics_3, val_metrics_3, test_metrics_3 = train(model, train_loader, None, test_loader, 30, optimizer, criterion, False)


Epoch: 1 Total_Time: 5.6274 Average_Time_per_batch: 0.0144 Train_Accuracy: 0.4585 Train_Loss: 1.4709 
Epoch: 2 Total_Time: 5.6231 Average_Time_per_batch: 0.0144 Train_Accuracy: 0.6430 Train_Loss: 1.0057 
Epoch: 3 Total_Time: 5.7164 Average_Time_per_batch: 0.0146 Train_Accuracy: 0.7265 Train_Loss: 0.7889 
Epoch: 4 Total_Time: 5.6963 Average_Time_per_batch: 0.0146 Train_Accuracy: 0.7708 Train_Loss: 0.6546 
Epoch: 5 Total_Time: 5.5621 Average_Time_per_batch: 0.0142 Train_Accuracy: 0.8076 Train_Loss: 0.5594 
Epoch: 6 Total_Time: 5.6155 Average_Time_per_batch: 0.0144 Train_Accuracy: 0.8303 Train_Loss: 0.5002 
Epoch: 7 Total_Time: 5.1917 Average_Time_per_batch: 0.0133 Train_Accuracy: 0.8515 Train_Loss: 0.4420 
Epoch: 8 Total_Time: 5.3789 Average_Time_per_batch: 0.0138 Train_Accuracy: 0.8640 Train_Loss: 0.3998 
Epoch: 9 Total_Time: 5.6819 Average_Time_per_batch: 0.0145 Train_Accuracy: 0.8735 Train_Loss: 0.3713 
Epoch: 10 Total_Time: 5.7081 Average_Time_per_batch: 0.0146 Train_Accuracy: 0.885

In [23]:
class ResNetDPN(nn.Module):
    def __init__(self, num_classes=10, mlp_hidden=256):
        super().__init__()
        # Load ResNet18 (pretrained=False for CIFAR10, since ImageNet weights use 224x224)
        backbone = torchvision.models.resnet18(weights=None)
        # Change input conv layer for CIFAR-10 (3x32x32)
        backbone.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        backbone.maxpool = nn.Identity()  # Remove the first maxpool
        # Extract up to the last layer
        self.backbone = nn.Sequential(*list(backbone.children())[:-1])  # Remove FC
        self.dpn = DPN_3(512, mlp_hidden + num_classes, num_classes, True).cuda()

    def forward(self, x):
        x = self.backbone(x)  # [B, 512, 1, 1]
        x = x.view(x.size(0), -1)  # Flatten
        x = self.dpn(x)
        return x

In [24]:
model = ResNetDPN(num_classes=10, mlp_hidden=256).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

In [25]:
train_metrics_3, val_metrics_3, test_metrics_3 = train(model, train_loader, None, test_loader, 30, optimizer, criterion, False)


Epoch: 1 Total_Time: 5.5823 Average_Time_per_batch: 0.0143 Train_Accuracy: 0.5086 Train_Loss: 1.3449 
Epoch: 2 Total_Time: 5.5892 Average_Time_per_batch: 0.0143 Train_Accuracy: 0.6929 Train_Loss: 0.8650 
Epoch: 3 Total_Time: 5.4887 Average_Time_per_batch: 0.0140 Train_Accuracy: 0.7638 Train_Loss: 0.6749 
Epoch: 4 Total_Time: 5.5511 Average_Time_per_batch: 0.0142 Train_Accuracy: 0.8030 Train_Loss: 0.5693 
Epoch: 5 Total_Time: 5.4835 Average_Time_per_batch: 0.0140 Train_Accuracy: 0.8288 Train_Loss: 0.4959 
Epoch: 6 Total_Time: 5.4471 Average_Time_per_batch: 0.0139 Train_Accuracy: 0.8487 Train_Loss: 0.4391 
Epoch: 7 Total_Time: 5.6113 Average_Time_per_batch: 0.0144 Train_Accuracy: 0.8639 Train_Loss: 0.3940 
Epoch: 8 Total_Time: 5.6505 Average_Time_per_batch: 0.0145 Train_Accuracy: 0.8756 Train_Loss: 0.3593 
Epoch: 9 Total_Time: 5.5383 Average_Time_per_batch: 0.0142 Train_Accuracy: 0.8852 Train_Loss: 0.3281 
Epoch: 10 Total_Time: 5.6667 Average_Time_per_batch: 0.0145 Train_Accuracy: 0.895

In [30]:
in_channels = 3                # change to 3 if you use CIFAR10 dataset
image_size = 32                # change to 32 if you use CIFAR10 dataset
num_classes = 10

lr = 4e-3
batch_size = 128
num_epochs = 10

patch_size = 16         # Each patch is 16x16, so 2x2 = 4 patches per image
hidden_dim = 768        # Token-mixing MLP hidden dim (formerly token_dim)
tokens_mlp_dim = 384    # Tokens MLP dim
channels_mlp_dim = 3072 # Channels MLP dim
num_blocks = 12         # Number of Mixer layers

In [ ]:
from MLP_Mixer import MLPMixer
model = MLPMixer(in_channels=in_channels, embedding_dim=channels_mlp_dim, num_classes=num_classes, patch_size=patch_size, image_size=image_size, depth=num_blocks, token_intermediate_dim=hidden_dim, channel_intermediate_dim=hidden_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

In [29]:
train_metrics_3, val_metrics_3, test_metrics_3 = train(model, train_loader, None, test_loader, 30, optimizer, criterion, False)


Epoch: 1 Total_Time: 4.5273 Average_Time_per_batch: 0.0116 Train_Accuracy: 0.3963 Train_Loss: 1.6372 
Epoch: 2 Total_Time: 4.9346 Average_Time_per_batch: 0.0126 Train_Accuracy: 0.5283 Train_Loss: 1.3032 
Epoch: 3 Total_Time: 5.3406 Average_Time_per_batch: 0.0137 Train_Accuracy: 0.5836 Train_Loss: 1.1585 
Epoch: 4 Total_Time: 4.8848 Average_Time_per_batch: 0.0125 Train_Accuracy: 0.6168 Train_Loss: 1.0675 
Epoch: 5 Total_Time: 5.5522 Average_Time_per_batch: 0.0142 Train_Accuracy: 0.6409 Train_Loss: 1.0070 
Epoch: 6 Total_Time: 4.5546 Average_Time_per_batch: 0.0116 Train_Accuracy: 0.6599 Train_Loss: 0.9537 
Epoch: 7 Total_Time: 4.5335 Average_Time_per_batch: 0.0116 Train_Accuracy: 0.6769 Train_Loss: 0.9060 
Epoch: 8 Total_Time: 5.0597 Average_Time_per_batch: 0.0129 Train_Accuracy: 0.6912 Train_Loss: 0.8665 
Epoch: 9 Total_Time: 5.4838 Average_Time_per_batch: 0.0140 Train_Accuracy: 0.7042 Train_Loss: 0.8396 
Epoch: 10 Total_Time: 4.9908 Average_Time_per_batch: 0.0128 Train_Accuracy: 0.713